In [16]:
import jax
import jax.numpy as jnp
from functools import partial


In [14]:
def f(x,val: int):
    return x.sum() + val




In [ ]:
jitted_f = jax.jit(f,static_argnames=["val"])

In [ ]:
out = jitted_f(jnp.array([1, 2, 3]), 10)